**Eletricity Data**
======================

**Set up Working Directory**
=============================

In [1]:
#Import packages
import pytz
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
import re

In [2]:
#Set up Working Directory and Data Files
drive = 'G'
if drive == 'G':
    wd = "/Users/dmatekenya/Google Drive/World-Bank/electricity_monitoring/"
    data_file = wd + '01.data/sms.csv'
else:
    wd = 'C:\\Users\\wb344850\\Box Sync\\01.GOKMU_BigDataPilot\\'
    data_file = wd + '01.data\\sms.csv'

In [3]:
#Read in Data
print ("Reading data.......")
x = pd.read_csv(data_file)

Reading data.......


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Drop unwanted columns
drop = ['sc_toa','read','status','locked', 'type',
        'subject','backup_date','protocol', 'date',
        'type', 'contact_name', 'service_center', 
        'backup_set']
x.drop(drop,inplace=True,axis=1)

In [5]:
#Check remaining columns
x.columns

Index(['count', 'address', 'body', 'toa', 'date_sent', 'readable_date'], dtype='object')

In [6]:
#Check the data
x.head()

,count,address,body,toa,date_sent,readable_date
0,144406,992935710546,Power Failure is running id 1301 | Monitoring...,null,1.476850e+12,10/19/2016 10:11:54
1,144406,992935710929,Power Failure is running id 1018 | Monitoring...,null,1.476850e+12,10/19/2016 10:12:07
2,144406,992935710544,Power Failure is running id 1063 | Monitoring...,null,1.476850e+12,10/19/2016 10:12:34
3,144406,992935710596,Power Failure is running id 1016 | Monitoring...,null,1.476850e+12,10/19/2016 10:14:22
4,144406,992935710849,Power Failure is running id 1043 | Monitoring...,null,1.476850e+12,10/19/2016 10:14:38


**So readable date refers to date_sent or date_received (which isnt available here) Whats this date for?**

In [12]:
#Check dates. For now just drop all bad dates (1906 records)
x['date_len'] = x.apply(lambda x: len(x['readable_date']),axis=1)

#Drop dates length > 19-bad dates
numrows_before = x.shape[0]
x = x[x.date_len==19]
print ("%s rows dropped due to bad date format" %(numrows_before-len(x)))

#create datetime_rcvd from readable date
x['datetime_rcvd'] = x.apply(lambda x: pd.datetime.strptime(x['readable_date'], '%m/%d/%Y %H:%M:%S'), axis=1)
#Drop date_len column
x.drop(['date_len'],inplace=True,axis=1)

0 rows dropped due to bad date format


In [13]:
#Remove rows without Device ID
msg = x.body
to_drop = []
device_id = {}
for i,b in enumerate(msg):
    str = re.findall(r'\d+', b)
    if len(str)<1:
         print (i,'-', b)
         to_drop.append(i)
    else:
        device_id [i] = str[0]
    

#Check the rows to be remnoved
print ("%s rows (shown below) dropped because they had no device-id info"%len(to_drop))
x.iloc[to_drop]

450 - Test SMS from Power Failure Monitor:Power Failure  is running ID
3165 - Power Back detected.
3167 - Power Back detected.
48055 - ??????? ????? ?????? ??????? ???. ??????? ?????? ??? ???????????
101737 - Test SMS from Power Failure Monitor:Power Failure detected. Il
141895 - ?????
141896 - ?????????
141899 - ??????? ????? ?????? ??????? ???. ??????? ?????? ??? ???????????
8 rows dropped because they had no device-id info


,count,address,body,toa,date_sent,readable_date,datetime_rcvd
450,144406,992935711044,Test SMS from Power Failure Monitor:Power Fail...,null,1.476880e+12,10/19/2016 17:10:08,2016-10-19 17:10:08
3165,144406,992935710834,Power Back detected.,null,1.477040e+12,10/21/2016 13:50:19,2016-10-21 13:50:19
3167,144406,992935710834,Power Back detected.,null,1.477040e+12,10/21/2016 13:51:07,2016-10-21 13:51:07
49961,144406,+992935821825,??????? ????? ?????? ??????? ???. ??????? ????...,null,1.482030e+12,12/18/2016 08:52:30,2016-12-18 08:52:30
103643,144406,992935710316,Test SMS from Power Failure Monitor:Power Fail...,null,1.486200e+12,02/04/2017 13:55:39,2017-02-04 13:55:39
143801,144406,938303656,?????,null,0.000000e+00,03/22/2017 16:57:26,2017-03-22 16:57:26
143802,144406,938303656,?????????,null,0.000000e+00,03/22/2017 16:57:43,2017-03-22 16:57:43
143805,144406,+992938303656,??????? ????? ?????? ??????? ???. ??????? ????...,null,1.490190e+12,03/22/2017 17:29:22,2017-03-22 17:29:22


In [14]:
#Drop the rows now and add deviucxe id
x.drop(x.index[to_drop],inplace=True)
df_device_id = pd.DataFrame.from_dict(device_id, orient='index')
df_device_id.columns = ['device_id']
#Join device id with original df
x = pd.concat([x,df_device_id],axis=1, join='inner')

In [103]:
#Type of Message
def detect_message(row):
    str = row ['body']
    fail = set(['Power','Failure','is','running'])
    if str[:4]=='Test':
        return 'test'
    elif set(str.split()[:4])==fail and 'Monitoring... Power OK' in str:
        return 'pon_mon'
    elif set(str.split()[:4])==fail and 'Monitoring... Power Failure' in str:
        return 'pfail_mon'
    elif set(str.split()[:4])==fail2 and 'Monitoring... Power OK' in str:
        return 'pon_mon'
    elif set(str.split()[:4])==fail2 and 'Monitoring... Power Failure' in str:
        return 'pfail_mon'
    elif str[:3]== 'Power Back detected':
         return 'pback'
    elif str[:22]== 'Power Failure detected':
        return 'pfail'
    elif str[:3]=='Power Failure' and 'Monitoring... Power Failure' in str:
        return 'pfail_mon'
    elif str[:3]=='Power Failure' and 'Monitoring... Power OK' in str:
        return 'pon_mon'
    else:
        return 'none'

In [94]:
#Generate message type variable
x['msg'] = x.apply(lambda x: detect_message(x),axis=1)
print ('Rows where I couldnt get the message %s'%len(x[x.msg=='none']))
#For now just drop rows with nessage=none
x = x[x.msg!='none']

In [107]:
#Drop rows where message is test
x = x[x.msg!='test']

In [108]:
#Generate variables
def message_type(msg):
    if msg == 'pback':
        return 1
    elif msg == 'pfail':
        return 2
    elif msg == 'pfail_mon':
        return 3
    elif msg == 'pon_mon':
        return 4

In [109]:
#Alive-variable to indicate whether box is alive or not
x["alive"] = ""
x.ix[x.msg=="pfail_mon", ['alive']] = 1
x.ix[x.msg=="pon_mon", ['alive']] = 1

#Message type-is the scale important here
x['msgty'] = x.apply(lambda row: message_type(row['msg']), axis = 1)

In [110]:
#Check duplicates and drop duplicates
print ("Before dropping duplicates....", x.shape)
x.drop_duplicates(subset=['device_id', 'body', 'readable_date'], keep = 'first', inplace=True)
print ("After dropping duplicates....", x.shape)

#print ("Before dropping duplicates....", x.shape)
x.drop_duplicates(subset=['device_id', 'datetime_rcvd'], keep = 'first', inplace=True)
print ("After dropping duplicates....", x.shape)

Before dropping duplicates.... (138582, 11)
After dropping duplicates.... (138561, 11)
After dropping duplicates.... (138561, 11)


In [111]:
#sort by device_id and datetime
x = x.sort_values(by =['device_id', 'datetime_rcvd'])

In [123]:
x[['device_id','datetime_rcvd', 'alive']].head()

,device_id,datetime_rcvd,alive
142083,0,2017-03-20 06:07:41,1
6950,1001,2016-10-27 03:12:55,1
7262,1001,2016-10-27 15:16:22,1
7511,1001,2016-10-28 03:16:26,1
7739,1001,2016-10-28 15:16:25,1


In [124]:
#create datetime-sent
stata_refdt = datetime.strptime("1/1/1960 00:00:00", '%m/%d/%Y %H:%M:%S')

#The constant 315637200000 is being added as some kind of offset based on stata code
x['datetime_sent'] = x.apply(lambda x: stata_refdt + timedelta(milliseconds = x['date_sent']+315637200000),axis=1)

In [125]:
#create delay variable
x['delay'] = x.apply(lambda x: (x['datetime_sent']-x['datetime_rcvd']).total_seconds(),axis=1)

In [126]:
#create time intervals between events
x['elapsed'] = (x['datetime_rcvd']-x['datetime_rcvd'].shift()).fillna(0)

In [127]:
x['elapsed_min'] = x.apply(lambda row: row['elapsed'].total_seconds()/60, axis=1)

In [128]:
#Replace elapsed = "" if alive =1
x.ix[x.alive==1, 'elapsed_min'] = np.NaN

In [129]:
#Sort by device_id and alive
x = x.sort_values(by =['device_id', 'alive'])

In [132]:
x['elapsed_min'].head()

142083   NaN
6950     NaN
7262     NaN
7511     NaN
7739     NaN
Name: elapsed_min, dtype: float64

In [ ]:
g elapsed_fail = elapsed if pback==1
g elapsed_on = elapsed if pfail==1

In [ ]:
#g elapsed_fail = elapsed if pback==
x.ix[x.msgty==2,'elapsed_fail'] = x2['elapsed_min']
x.ix[x.msgty==1,'elapsed_on'] = x2['elapsed_min']

In [137]:
#Sort by device_id and alive
x = x.sort_values(by =['device_id', 'datetime_rcvd'])

In [138]:
x[['device_id','elapsed_min','datetime_rcvd']].head()

,device_id,elapsed_min,datetime_rcvd
142083,0,NaN,2017-03-20 06:07:41
6950,1001,NaN,2016-10-27 03:12:55
7262,1001,NaN,2016-10-27 15:16:22
7511,1001,NaN,2016-10-28 03:16:26
7739,1001,NaN,2016-10-28 15:16:25


In [ ]:
x[x.msgty==2].head()